<a href="https://colab.research.google.com/github/syedshubha/TeachingQuantumComputing/blob/main/QKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet qiskit qiskit-aer pylatexenc qiskit_ibm_runtime &> /dev/null

In [2]:
from qiskit import *
from qiskit_aer import Aer
from qiskit_aer.primitives import Estimator
from qiskit.quantum_info import Pauli

In [3]:
sim = Aer.get_backend('aer_simulator')
est = Estimator()
shots=100000

In [4]:
q = QuantumCircuit(1)
q.h(0)

expected_value = est.run(q, Pauli('Z'), shots=shots).result().values[0]
print(expected_value)
q.measure_all()
counts = sim.run(q, shots=shots).result().get_counts()
print(counts)

-0.00608
{'0': 50323, '1': 49677}


# B92

In [5]:
def alice_encoding(bits):
  states = []
  for i in range(n):
    q = QuantumCircuit(1,1)
    if bits[i] ==1:
      q.h(0)
    states.append(q)
  return states

In [6]:
def eve_eavesdropping(basis, states):
  for i in range(n):
    if basis[i] == 1:
      states[i].h(0)
    states[i].measure(0,0)
  return states

In [7]:
def bob_decoding(basis, states):
  bob_key = ""
  public = []
  for i in range(n):
    if basis[i] == 1:
      states[i].h(0)
    states[i].measure(0,0)
    counts = sim.run(states[i], shots=1).result().get_counts()
    if  list(counts.keys())[0] == '1':
      public.append(i)
      bob_key = bob_key + str(1-basis[i])
  return bob_key, public

In [8]:
def classical_comm(public,alice_bits):
  alice_key = ""
  for i in public:
    alice_key = alice_key + str(alice_bits[i])
  return alice_key

In [9]:
from random import choices
n = 100
alice_bits = choices([0, 1], k=n)
Q = alice_encoding(alice_bits)

bob_basis = choices([0, 1], k=n)
bob_key, public = bob_decoding(bob_basis, Q)
alice_key = classical_comm(public,alice_bits)

print(alice_key)
print(bob_key)
print(alice_key == bob_key)

0101111010001010111111001101
0101111010001010111111001101
True


In [10]:
#EavesDropping

alice_bits = choices([0, 1], k=n)
Q = alice_encoding(alice_bits)

eve_basis = choices([0, 1], k=n)
Q = eve_eavesdropping(eve_basis, Q)

bob_basis = choices([0, 1], k=n)
bob_key, public = bob_decoding(bob_basis, Q)
alice_key = classical_comm(public,alice_bits)

print(alice_key)
print(bob_key)
print(alice_key == bob_key)

1101010111011111001010110010101010011
1101000100110000110011100001100110100
False


# CHSH Inequality

In [11]:
from itertools import product
S = []
for a1,a2,b1,b2 in product([-1,1], repeat=4):
  s = a1*(b1+b2) + a2*(b1-b2)
  if s not in S:
    S.append(s)
print(S)

[2, -2]


In [12]:
q = QuantumCircuit(2)
q.h(0)
q.h(1)

observable_xy = Pauli('XX')

XX = est.run(q, Pauli('XX'), shots=shots).result().values[0]
ZZ = est.run(q, Pauli('ZZ'), shots=shots).result().values[0]

CHSH = abs((2**0.5)*(XX+ZZ))
print(CHSH)

1.4129124858957118


In [13]:
q = QuantumCircuit(2)
q.h(0)
q.cx(0,1)

observable_xy = Pauli('XX')

XX = est.run(q, Pauli('XX'), shots=shots).result().values[0]
ZZ = est.run(q, Pauli('ZZ'), shots=shots).result().values[0]

CHSH = abs((2**0.5)*(XX+ZZ))
print(CHSH)

2.8284271247461903
